# Analyse the GA results

In [94]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

%matplotlib widget

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [95]:
import os
import numpy as np
import pandas as pd
import logging
import pickle
import jupyter_black
from tqdm import tqdm

from src.data_connectors import write_solution_files
from src.data_connectors import read_input_files
from src.data_connectors import read_solutions_files
from src.genetic_algorithm import constants

jupyter_black.load()

logging.basicConfig()
logging.getLogger().setLevel(logging.INFO)

## Read the results

In [96]:
# filename = "../data/solutions/all_ga_MMTSP_SAC_swap.pkl"
# filename = "../data/solutions/all_ga_MMTSP_SAC_sorted_paretns.pkl"
filename = "../data/solutions/all_ga_MMTSP_SAC_best.pkl"
filename = "../data/solutions/all_ga_MMTSP_SAC.pkl"
with open(filename, "rb") as f:
    loaded_results = pickle.load(f)

In [97]:
len(loaded_results)

120

In [98]:
# First instance
loaded_results[0][-1]

217

In [99]:
# Last instance
loaded_results[-1][-1]

336

### Get Latency results

In [100]:
latencies = []
for instance_number in constants.INSTANCES_LIST:
    latency = read_solutions_files.read_latency_from_solutions_header(
        filename=f"../data/solutions/MMTSP-SAC/solution_{instance_number}.txt"
    )
    latencies.append(latency)

## Process results

list[all_surviving_chromosomes, all_surviving_makespans, best_makespan, duration_until_best_makespan]

In [101]:
instances_path = "../data/input/HRTInstances/"
instances_list = constants.INSTANCES_LIST

times_of_best_results = []

for best, instance_number in tqdm(zip(loaded_results, instances_list)):
    ins_x = ins_x = read_input_files.read_file(
        os.path.join(instances_path, f"Instance_{instance_number}.txt")
    )
    times_of_best_results.append(write_solution_files.find_times_of_best_solution(ins_x, best))

120it [00:18,  6.55it/s]


In [102]:
# Number of isntances
len(times_of_best_results)

120

In [103]:
# Structure of resuts
times_of_best_results[0]

{'0': {2: (2, 37),
  22: (38, 73),
  3: (74, 79),
  23: (80, 85),
  4: (86, 116),
  24: (117, 147),
  5: (148, 247),
  25: (248, 347),
  6: (348, 384),
  26: (385, 421),
  7: (422, 463),
  27: (464, 505),
  9: (506, 526),
  29: (527, 547),
  10: (548, 646),
  30: (647, 745),
  12: (746, 790),
  32: (791, 835),
  13: (836, 883),
  33: (884, 931),
  14: (932, 969),
  34: (970, 1007),
  15: (1008, 1069),
  35: (1070, 1131),
  17: (1132, 1162),
  37: (1163, 1193),
  18: (1194, 1221),
  38: (1222, 1249),
  20: (1250, 1338),
  40: (1339, 1427)},
 '1': {1: (0, 1), 8: (38, 48), 11: (49, 54), 16: (791, 796), 19: (797, 891)},
 '2': {21: (0, 1), 28: (74, 84), 31: (85, 90), 36: (836, 841), 39: (842, 936)}}

## Optimal Solutions

In [104]:
df_optimal = pd.read_csv("../data/solutions/optimal/OptimalSolutions.csv", sep=";", header=1)

df_optimal.head()

,Instance,Humans,Robots,Robot Eligibility,Optimal solution,Best known solution,Lower bound
0,217,1,2,"0,25",1397.0,1397,1397
1,218,1,2,"0,5",865.0,865,865
2,219,1,2,1,529.0,529,529
3,220,1,2,"0,25",3316.0,3316,3316
4,221,1,2,"0,5",2476.0,2476,2476


In [105]:
def get_gap(df_optimal: pd.DataFrame, instance_number: int, best: tuple) -> float:
    lower_bound = df_optimal[df_optimal.Instance == instance_number]["Lower bound"].values[0]
    ins_result = best[2]
    gap = (ins_result + 1 - lower_bound) / lower_bound * 100
    logging.debug(f"Lower Bound: {lower_bound}, Result {ins_result}")
    logging.debug(f"Instance {instance_number} has Gap % = {gap:.2f} %")
    return gap

In [107]:
instances_list = [x for x in range(217, 336)]

pct_deviation_from_best = []

for best in tqdm(loaded_results):
    instance_number = best[-1]
    optimal = df_optimal[df_optimal.Instance == instance_number]["Optimal solution"].values[0]
    best_know = df_optimal[df_optimal.Instance == instance_number]["Best known solution"].values[0]
    logging.debug(f"Optimal: {optimal}, Best Known: {best_know}")
    gap = get_gap(df_optimal, instance_number, best)
    logging.debug(f"Instance {instance_number} has Gap % = {gap:.2f} %")

100%|██████████| 120/120 [00:00<00:00, 3211.20it/s]


## Group by type

In [108]:
df_instance_groups = (
    df_optimal.groupby(["Humans", "Robots", "Robot Eligibility"]).Instance.apply(list).reset_index()
)
df_instance_groups


all_gaps = []
all_gaps_sequence = []
all_is_optimal = []
all_lower_gaps = []
all_ids_lower_gaps = []
for idx, row_group in tqdm(df_instance_groups.iterrows()):
    gap_group = []
    is_optimal_group = []
    lower_gaps = []
    ids_lower_gaps = []
    for best in tqdm(loaded_results):
        instance_number = best[-1]
        if instance_number in row_group.Instance:
            gap_x = get_gap(df_optimal, instance_number, best)
            gap_group.append(gap_x)
            if gap_x <= 0:
                is_optimal_group.append(1)
            else:
                is_optimal_group.append(0)
            if gap_x <= 10:
                lower_gaps.append(1)
                ids_lower_gaps.append(instance_number)
            else:
                lower_gaps.append(0)
                ids_lower_gaps.append(0)

    all_gaps.append(gap_group)
    all_gaps_sequence.extend(gap_group)
    all_is_optimal.append(is_optimal_group)
    all_lower_gaps.append(lower_gaps)
    all_ids_lower_gaps.extend(ids_lower_gaps)

df_instance_groups["Gaps"] = all_gaps
df_instance_groups["optimals"] = all_is_optimal
df_instance_groups["lower_gaps"] = all_lower_gaps
df_instance_groups["Gaps%"] = round(df_instance_groups["Gaps"].apply(lambda x: sum(x) / len(x)), 2)
df_instance_groups["# optimals"] = df_instance_groups["optimals"].apply(lambda x: sum(x))
df_instance_groups["# all_lower_gaps"] = df_instance_groups["lower_gaps"].apply(lambda x: sum(x))

100%|██████████| 120/120 [00:00<00:00, 41194.67it/s]
12it [00:00, 125.03it/s]


In [123]:
# Mean Gaps
np.mean(all_gaps)

37.59125701860223

In [110]:
# STD Gaps
np.std(all_gaps)

25.291489276746727

In [122]:
# Results
df_instance_groups

,Humans,Robots,Robot Eligibility,Instance,Gaps,optimals,lower_gaps,Gaps%,# optimals,# all_lower_gaps
0,1,2,"0,25","[217, 220, 229, 232, 241, 244, 253, 256, 265, ...","[2.219040801717967, 2.2919179734620023, 10.158...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 0, 0, 0, 0, 0, 0, 0, 0]",17.59,0,2
1,1,2,"0,5","[218, 221, 230, 233, 242, 245, 254, 257, 266, ...","[13.0635838150289, 5.815831987075929, 10.17119...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 1, 0, 1, 1, 0, 0, 0, 0, 0]",19.05,0,3
2,1,2,1,"[219, 222, 231, 234, 243, 246, 255, 258, 267, ...","[22.495274102079396, 50.98305084745762, 22.489...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",43.59,0,0
3,1,3,"0,25","[223, 226, 235, 238, 247, 250, 259, 262, 271, ...","[1.76678445229682, 2.3399014778325125, 8.03149...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 0, 1, 0, 1, 0, 1]",10.11,0,7
4,1,3,"0,5","[224, 227, 236, 239, 248, 251, 260, 263, 272, ...","[1.7377567140600316, 8.277027027027026, 4.4871...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 0, 0, 0, 0, 0, 0, 1]",20.47,0,4
5,1,3,1,"[225, 228, 237, 240, 249, 252, 261, 264, 273, ...","[21.354933726067745, 44.74532559638942, 34.157...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",48.16,0,0
6,2,2,"0,25","[277, 280, 289, 292, 301, 304, 313, 316, 325, ...","[13.274336283185843, 16.735324407826983, 18.38...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",30.50,0,0
7,2,2,"0,5","[278, 281, 290, 293, 302, 305, 314, 317, 326, ...","[8.362369337979095, 45.53651938683498, 20.6806...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]",45.89,0,1
8,2,2,1,"[279, 282, 291, 294, 303, 306, 315, 318, 327, ...","[43.53741496598639, 85.25714285714285, 76.1682...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",66.47,0,0
9,2,3,"0,25","[283, 286, 295, 298, 307, 310, 319, 322, 331, ...","[3.821656050955414, 14.911952041963284, 35.616...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]",31.04,0,1


In [112]:
# Number of instances 
df_instance_groups["# all_lower_gaps"].sum()

18

In [113]:
# Mean latency (per instance) 
np.mean(latencies)

210.40291666666664

In [114]:
# Mean Latency (per instance) in minutes
np.mean(latencies) / 60

3.5067152777777775

In [115]:
# Total latency in houra
np.sum(latencies) / 60 / 60

7.013430555555555

In [116]:
# Total instances with results in less than 1 minute
len([x for x in latencies if x < 60])

21

In [126]:
# Instance with lower gaps
all_lower_gaps_than_10 = [x for x, y in zip(all_gaps_sequence, all_ids_lower_gaps) if y != 0]
# Number of instance with lower gaps
all_ids_lower_gaps = [x for x in all_ids_lower_gaps if x > 0]
len(all_ids_lower_gaps)

18

In [128]:
# Mean value of instances with lower gap
np.mean(all_lower_gaps_than_10)

5.993867694116094